In [1]:
# to access root modules
if __name__ == "__main__" and __package__ is None:
    from sys import path
    from os.path import dirname as dir

    path.append(dir(path[0]))
    __package__ = "dash_app"

In [2]:
import pandas as pd
import numpy as np
import datetime
import dateutil
import plotly.graph_objects as go
from plotly.subplots import make_subplots
from charts import stations_map, map_columns,percentile_series
from settings import *

In [3]:
data_path = DATA_PATH+'Oceanic_Domain/3.9OceanColour/Figure3.20/'
data_csv = data_path + 'Figure3.20_data.csv'

In [4]:
"""
Tidy data for charts
"""
columns_dict = {'datetime':'(month)',
               'mean__monthly__mass_concentration_of_chlorophyll_a_in_sea_water':'Monthly mean (46º-58º N, 5º-26º W)',
                  }
xls = pd.ExcelFile(
    data_path+'Monthly_Mean_ChlorophyllA_Concentration_IrishMaritimeBoundaryContinentalShelf.xlsx')
original_df = pd.read_excel(xls, 'SWFMO_CHLO.CR.l3m_data.Oct1997-',skiprows=8)
df = map_columns(columns_dict, original_df)
df['datetime'] = pd.to_datetime(df['datetime'],format='%Y-%m-%d %H:%M:%S')
#df.to_csv(data_csv) #commented out as the file should be kept as is

In [3]:
data_path = DATA_PATH+'Oceanic_Domain/3.9OceanColour/Figure3.20/'
data_csv = data_path + 'Figure3.20_data.csv'
df = pd.read_csv(data_csv, index_col=0)
df['datetime'] = pd.to_datetime(df['datetime'])
df['percentile'] = percentile_series(df,'mean__monthly__mass_concentration_of_chlorophyll_a_in_sea_water')

In [9]:

trace = go.Heatmap(
    z=df['percentile'],
    x=df['datetime'].dt.month,
    y=df['datetime'].dt.year,
    text=df['mean__monthly__mass_concentration_of_chlorophyll_a_in_sea_water'],
    colorscale=PERCENTILE_COLORSCALE, 
    colorbar=dict(
                tickmode='array',
                thickness=10,
                len=0.9,
        
    title='<b>Percentile</b> (mg/m\u00B3)',
            ticktext=[
                '<b>Min.</b> (0.56)', 
                '<b> 5%</b> (0.6)', 
                '<b>25%</b> (0.66)', 
                '<b>50%</b> (0.73)',
                '<b>75%</b> (0.81)', 
                '<b>95%</b> (0.92)'],
            tickvals=[0,5,24,47, 71, 90]  
    ),
    hovertemplate='%{x} %{y}<br>'+
    'Concentration: %{text:.2f} mg/m\u00B3<extra></extra>'
)
figure_3_20 = go.Figure(data=trace, layout=TIMESERIES_LAYOUT)
figure_3_20.update_layout(
yaxis=dict(
    title='Year',
    nticks=12),
xaxis=dict(
    title='Month',
    ticktext=['Jan','Feb','Mar','Apr','May','June','July','Aug','Sep','Oct','Nov','Dec'],
    showgrid=False,
   tickvals=[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12],
))

# Include white lines as xaxis gridlines
x0=1.5
for i in range(0,11):
    figure_3_20.add_shape(type='line',
                        x0=x0, y0=2001.5, x1=x0, y1=2019,
                        line=dict(color='White', width=3))
    x0+=1

figure_3_20